# Clustering Toronto Neighbourhoods

## Load libraries

In [18]:
import requests

import pandas as pd
import folium

## Data Exctraction and Cleaning

Toronto neighbourhood information was extractred from [Wikipedia](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) directly using the pandas read_html funtion.

In [19]:
# Extract table from wikipedia using Pandas
table_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

# Remove unassigned boroughs
table_df = table_df[table_df.Borough != 'Not assigned'].reset_index(drop=True)

table_df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Above we can see, we have the correct columns and the first 5 rows of the data frame look correct. 

## Data checking and verifying

The DataFrame was checked to make sure no post codes were shared and that all had neighbourhood(s) assigned. Finally the shape of the Dataframe was shown. 

In [20]:
table_df['Postal Code'].value_counts().value_counts()

1    103
Name: Postal Code, dtype: int64

Here we can see we have 103 distinct Postal Codes.

In [21]:
len(table_df[table_df.Neighbourhood != 'Not assigned'])

103

Here we can see we have 103 distinct postcodes with neighbourhood(s) assigned.

In [22]:
table_df.shape

(103, 3)

Our dataframe has 103 rows and 3 columns. From above we can deduce that there are no duplicate post codes in the table, nor do any post codes have no assigned neighbourhood(s)

## Adding latitude and the longitude coordinates

The Geospatial_Coordinates.csv provided was used and read through Pandas and combined with table extracted from Wikipedia

In [23]:
GeoCoords = pd.read_csv('https://cocl.us/Geospatial_data')

In [24]:
tor_coord = table_df.merge(GeoCoords,how='right',on='Postal Code')

In [25]:
tor_coord.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Neighbourhood Clusters

All neighbourhoods containing 'Toronto' in their title were mapped.

In [26]:
# Get Boroughs only with 'Toronto' in name
tor_boro = tor_coord[tor_coord.Borough.str.contains("Toronto", regex = False)]

In [27]:
#Get central coordinates for Toronto map
latitude = tor_boro.Latitude.mean()
longitude = tor_boro.Longitude.mean()

In [28]:
# Create map object for Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# Add map markers for each neighbourhood
for lat, lng, borough, neighborhood in zip(tor_boro.Latitude, tor_boro.Longitude, tor_boro.Borough, tor_boro.Neighbourhood):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

## Exploring the Neighbourhoods

In [29]:
CLIENT_ID = 'JHD12LNKPLUI4FCOPN1Q0QAZS2CCIYWXXXYHUBOOYPD3LBRR' # your Foursquare ID
CLIENT_SECRET = 'L1TC5ELHY2RAETCJOCMOF3OFA4KHWLXIHOJDFYNU0NNG14KS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
radius = 500

Using the getNearbyVenues function from the Segmenting and Clustering Neighborhoods in New York City Lab:

In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
xyz = getNearbyVenues(tor_boro.Neighbourhood, tor_boro.Longitude, tor_boro.Latitude)

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

ValueError: Length mismatch: Expected axis has 0 elements, new values have 7 elements

In [35]:
tor_boro.iloc[-1]

Postal Code                                                    M7Y
Borough                                               East Toronto
Neighbourhood    Business reply mail Processing Centre, South C...
Latitude                                                 43.662744
Longitude                                               -79.321558
Name: 87, dtype: object

In [36]:
xyz

NameError: name 'xyz' is not defined